In [1]:
import json
import numpy
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

In [2]:
def clean_report_mimic_cxr(report):
    report_cleaner = lambda t: t.replace('\n', ' ').replace('__', '_').replace('__', '_').replace('__', '_') \
        .replace('__', '_').replace('__', '_').replace('__', '_').replace('__', '_').replace('  ', ' ') \
        .replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ') \
        .replace('..', '.').replace('..', '.').replace('..', '.').replace('..', '.').replace('..', '.') \
        .replace('..', '.').replace('..', '.').replace('..', '.').replace('1. ', '').replace('. 2. ', '. ') \
        .replace('. 3. ', '. ').replace('. 4. ', '. ').replace('. 5. ', '. ').replace(' 2. ', '. ') \
        .replace(' 3. ', '. ').replace(' 4. ', '. ').replace(' 5. ', '. ') \
        .strip().lower().split('. ')
    sent_cleaner = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '')
                                    .replace('\\', '').replace("'", '').strip().lower())
    tokens = [sent_cleaner(sent) for sent in report_cleaner(report) if sent_cleaner(sent) != []]
    report = ' . '.join(tokens) + ' .'
    return report

In [3]:
with open('../data/kg_node.json') as f:
    data = json.load(f)

In [4]:
data

{'normal': 0,
 'pleural': 220661,
 'effusion': 204633,
 'pneumothorax': 198977,
 'lung': 119446,
 'lungs': 117604,
 'pulmonary': 110935,
 'consolidation': 107340,
 'mediastinal': 97515,
 'silhouette': 95344,
 'contours': 93225,
 'cardiomediastinal': 85736,
 'heart': 81456,
 'hilar': 78590,
 'cardiac': 70955,
 'atelectasis': 68046,
 'edema': 63375,
 'osseous': 50848,
 'opacity': 46192,
 'pneumonia': 37637,
 'effusions': 36617,
 'opacities': 34646,
 'lobe': 33617,
 'vascular': 31501,
 'thoracic': 30851,
 'hemidiaphragm': 25828,
 'aorta': 25711,
 'vasculature': 22911,
 'cardiomegaly': 22770,
 'spine': 20913,
 'silhouettes': 19480,
 'rib': 17219,
 'retrocardiac': 11921,
 'opacification': 11338,
 'fracture': 10870,
 'costophrenic': 10535,
 'atrium': 10384,
 'fractures': 10248,
 'endotracheal': 9537,
 'stomach': 9286,
 'calcifications': 8856,
 'ventricle': 7767,
 'venous': 7719,
 'thickening': 7297,
 'airspace': 6309,
 'vertebral': 5258,
 'perihilar': 5186,
 'emphysema': 4949,
 'calcificatio

In [5]:
len(data)

124

In [33]:
pair = []
for i, (sk, sv) in enumerate(data.items()):
    for j, (dk, dv) in enumerate(data.items()):
#         if j <= i:
#             continue
#         else:
        pair.append([sk, dk])

In [34]:
len(pair), pair[:2]

(15376, [['normal', 'normal'], ['normal', 'pleural']])

In [35]:
co_freq = {}
for sk, dk in pair:
    co_freq[sk + '|' + dk] = 0

In [9]:
with open('/home/shuxinyang/data/mimic/xrv/annotation.json', 'r') as f:
    corpus = json.load(f)

In [10]:
train = corpus['train']
train[0]

{'id': '02aa804e-bde0afdd-112c0b34-7bc16630-4e384014',
 'study_id': 50414267,
 'subject_id': 10000032,
 'report': 'There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral\n nodular opacities that most likely represent nipple shadows. The\n cardiomediastinal silhouette is normal.  Clips project over the left lung,\n potentially within the breast. The imaged upper abdomen is unremarkable.\n Chronic deformity of the posterior left sixth and seventh ribs are noted.',
 'image_path': ['p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg'],
 'split': 'train'}

In [36]:
for item in tqdm(train):
    report = clean_report_mimic_cxr(item['report'])
    for sk, dk in pair:
        if sk in report and dk in report:
            co_freq[sk + '|' + dk] += 1

100%|██████████| 270790/270790 [12:56<00:00, 348.71it/s]


In [37]:
with open('../data/co_occurrence_frequency.json', 'w') as f:
    json.dump(co_freq, f);

In [38]:
freqs = []
for k, v in co_freq.items():
    freqs.append(v)
freqs.sort()
max(freqs)

226765

In [39]:
n = len(freqs)
split_num = 5 + 1
for i in range(1, split_num):
    print(i, ': ', freqs[int(n * (i / split_num))])

partition = []
for i in range(1, split_num):
    partition.append(freqs[int(n * (i / split_num))])
partition.append(freqs[-1] + 1)

1 :  9
2 :  35
3 :  127
4 :  449
5 :  1617


In [40]:
partition

[9, 35, 127, 449, 1617, 226766]

In [41]:
knowlegde_graph = {'head': [], 'relation': [], 'tail': []}
for k, v in tqdm(co_freq.items()):
    if v == 0:
        continue
    sk, dk = k.split('|')
    knowlegde_graph['head'].append(sk)
    knowlegde_graph['tail'].append(dk)
    for level, p in enumerate(partition[1:]):
        if v <= p:
            knowlegde_graph['relation'].append(level + 1)
            break

100%|██████████| 15376/15376 [00:00<00:00, 612380.41it/s]


In [42]:
# knowlegde_graph['relation'] = [len(knowlegde_graph['relation'])] + knowlegde_graph['relation']
# knowlegde_graph['head'] = [len(knowlegde_graph['relation'])] + knowlegde_graph['head']
# knowlegde_graph['tail'] = [len(knowlegde_graph['relation'])] + knowlegde_graph['tail']

df = pd.DataFrame.from_dict(knowlegde_graph)

In [43]:
max(knowlegde_graph['relation']), min(knowlegde_graph['relation']), len(knowlegde_graph['relation'])

(5, 1, 14464)

In [44]:
# df.to_csv('../data/MIMIC_KG.csv', index=False, header=False)
df.to_csv('../data/MIMIC_KG.tsv', index=False, header=False, sep='\t')

In [45]:
df

,head,relation,tail
0,normal,5,normal
1,normal,5,pleural
2,normal,5,effusion
3,normal,5,pneumothorax
4,normal,5,lung
...,...,...,...
14459,cicatricial,1,thorax
14460,cicatricial,1,hemidiaphragms
14461,cicatricial,1,bronchiectasis
14462,cicatricial,1,pleura


In [80]:
# 只用 大于5 的关系
knowlegde_graph = {'head': [], 'relation': [], 'tail': []}
for k, v in tqdm(co_freq.items()):
    if v < partition[5]:
        continue
    sk, dk = k.split('|')
    knowlegde_graph['head'].append(sk)
    knowlegde_graph['tail'].append(dk)
    for level, p in enumerate(partition[5:]):
        if v <= p:
            knowlegde_graph['relation'].append(level + 1)
            break

100%|██████████| 7626/7626 [00:00<00:00, 505983.74it/s]


In [81]:
df = pd.DataFrame.from_dict(knowlegde_graph)
max(knowlegde_graph['relation']), min(knowlegde_graph['relation']), len(knowlegde_graph['relation'])

(5, 1, 3053)

In [82]:
df

,head,relation,tail
0,normal,5,pleural
1,normal,5,effusion
2,normal,5,pneumothorax
3,normal,5,lung
4,normal,5,lungs
...,...,...,...
3048,pleura,2,fibrotic
3049,pleura,3,pneumoperitoneum
3050,pleura,2,shadows
3051,pleura,4,sclerotic


In [83]:
df.to_csv('../data/KGE_train_great5.csv', index=False, header=False)
df.to_csv('../data/KGE_train_great5.tsv', index=False, header=False, sep='\t')

In [84]:
import random
idx = [i for i in range(3053)]

In [93]:
from sklearn.model_selection import train_test_split
sample_id = train_test_split(idx, test_size=0.2)
len(sample_id[0]), len(sample_id[1])

In [102]:
train_head = np.asarray(knowlegde_graph['head'])[sample_id[0]]
train_relation = np.asarray(knowlegde_graph['relation'])[sample_id[0]]
train_tail = np.asarray(knowlegde_graph['tail'])[sample_id[0]]

In [105]:
df = pd.DataFrame.from_dict({'head': train_head, 'relation': train_relation, 'tail': train_tail})
df.to_csv('../data/KGE_train_split_great5.tsv', index=False, header=False, sep='\t')

In [106]:
test_head = np.asarray(knowlegde_graph['head'])[sample_id[1]]
test_relation = np.asarray(knowlegde_graph['relation'])[sample_id[1]]
test_tail = np.asarray(knowlegde_graph['tail'])[sample_id[1]]

In [107]:
df = pd.DataFrame.from_dict({'head': test_head, 'relation': test_relation, 'tail': test_tail})
df.to_csv('../data/KGE_test_split_great5.tsv', index=False, header=False, sep='\t')